In [88]:
#import all packages
import pandas as pd
import numpy as np
import re as re
import nltk
from nltk.stem.porter import *

from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

import nltk.classify.util
from nltk.classify import NaiveBayesClassifier

from datetime import datetime, timedelta

In [89]:
def reduce_lengthening(text):
    pattern = re.compile(r"(.)\1{2,}")
    return pattern.sub(r"\1\1", text)

In [90]:
# datafile
data_file = "data_with_topic.csv"
data = pd.read_csv(data_file)
data.columns = data.columns.str.strip().str.replace(" ","_") # reason for doing this is cause cannot get columns with space into a list

#reformat the respective columns

#lower case
data['Review'] = [word.lower() for word in data['Review']]

#Strip off the Variation Word
data['Product_Purchase'] = data['Product_Purchase'].str.strip("Variation:")

#Change Date of Review to DateTime
#data['Date_Of_Review'] = pd.to_datetime(data['Date_Of_Review'])

data.info()
data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9002 entries, 0 to 9001
Data columns (total 14 columns):
Unnamed:_0          9002 non-null int64
Platform            9002 non-null object
Brand               9002 non-null object
Category            9002 non-null object
Product_Name        9002 non-null object
Price               9002 non-null float64
Reviewer            8980 non-null object
Review              9002 non-null object
Product_Purchase    7394 non-null object
Ratings             8980 non-null float64
Date_Of_Review      8980 non-null object
Response            8980 non-null object
Dorminant_Topic     9002 non-null float64
topic               9002 non-null object
dtypes: float64(3), int64(1), object(10)
memory usage: 984.7+ KB


,Unnamed:_0,Platform,Brand,Category,Product_Name,Price,Reviewer,Review,Product_Purchase,Ratings,Date_Of_Review,Response,Dorminant_Topic,topic
0,0,Shopee,L'Oreal Paris,Serum & Treatment,L'Oreal Paris Revitalift Crystal Micro-Essence...,25.9,zenheng,"delivery was quite fast, and item was on disco...",NaN,4.0,2020-02-06 23:58,no,3.0,delivery
1,1,Shopee,L'Oreal Paris,Serum & Treatment,L'Oreal Paris Revitalift Crystal Micro-Essence...,25.9,glennoeestaquio,received item sealed properly.\r\r\r\nquick de...,NaN,5.0,2020-02-06 12:44,no,3.0,delivery
2,2,Shopee,L'Oreal Paris,Serum & Treatment,L'Oreal Paris Revitalift Crystal Micro-Essence...,25.9,shy2206,yet to try but delivery was prompt and product...,NaN,5.0,2020-02-06 10:21,no,3.0,delivery
3,3,Shopee,L'Oreal Paris,Serum & Treatment,L'Oreal Paris Revitalift Crystal Micro-Essence...,25.9,lindamiyalin,"well wrapped, in good condition, super fast ...",NaN,5.0,2020-01-30 23:01,no,0.0,product
4,4,Shopee,L'Oreal Paris,Serum & Treatment,L'Oreal Paris Revitalift Crystal Micro-Essence...,25.9,kyc385,"well packed, will buy again",NaN,5.0,2020-02-01 11:40,no,3.0,delivery


### Transformation and Initalise of data

In [91]:
# make the old csv data into a list

row_id_list = data['Unnamed:_0'].tolist()
platform_list = data['Platform'].tolist()
brand_list = data['Brand'].tolist()
category_list = data['Category'].tolist()
product_name_list = data['Product_Name'].tolist()
price_list = data['Price'].tolist()
reviewer_list = data['Reviewer'].tolist()
review_list = data['Review'].tolist()
product_variation_list = data['Product_Purchase'].tolist()
rating_list = data['Ratings'].tolist()
date_review_list = data["Date_Of_Review"].tolist()
response_list = data["Response"].tolist()

# initialising the new columns

id_csv=[]
platform_csv=[]
brand_csv = []
category_csv = []
product_name_csv = []
prices_csv = []
reviewer_csv =[]
review_splitted_csv = []
review_csv=[]
product_variation_csv = []
rating_csv = []
date_review_csv = []
response_csv = []

# this is the length of the csv old data
print ("The length of the dataset is:", len(brand_list))

The length of the dataset is: 9002


In [92]:
#upload the appos
from word_dict import appos

print(appos)

{"aren't": 'are not', "can't": 'cannot', "couldn't": 'could not', "didn't": 'did not', "doesn't": 'does not', "don't": 'do not', "hadn't": 'had not', "hasn't": 'has not', "haven't": 'have not', "he'd": 'he would', "he'll": 'he will', "he's": 'he is', "i'd": 'I had', "i'll": 'I will', "i'm": 'I am', "isn't": 'is not', "it's": 'it is', "it'll": 'it will', "i've": 'I have', "let's": 'let us', "l'oreal": 'loreal', "mightn't": 'might not', "mustn't": 'must not', "shan't": 'shall not', "she'd": 'she would', "she'll": 'she will', "she's": 'she is', "shouldn't": 'should not', "that's": 'that is', "there's": 'there is', "they'd": 'they would', "they'll": 'they will', "they're": 'they are', "they've": 'they have', "we'd": 'we would', "we're": 'we are', "weren't": 'were not', "we've": 'we have', "what'll": 'what will', "what're": 'what are', "what's": 'what is', "what've": 'what have', "where's": 'where is', "who'd": 'who would', "who'll": 'who will', "who're": 'who are', "who's": 'who is', "who'

### Spilt the Sentence 

In [ ]:
import itertools
from spellchecker import SpellChecker
from nltk.tokenize import word_tokenize

spell = SpellChecker()

for i in range(len(brand_list)):
    # current_review is a string
    current_review = review_list[i]
    #print(current_review)
    
    
    date_review =date_review_list[i]
    if(platform_list[i] == 'Lazada'):
        #Transform the date format
        
        dt= datetime.today()

        if 'days ago' in date_review or 'day ago' in date_review :
            day = date_review.replace('days ago', '').replace('day ago', '')
            dt = datetime.today() - timedelta(days=int(day))

        elif 'weeks ago' in date_review or 'week ago' in date_review:
            day = date_review.replace('weeks ago', '').replace('week ago', '')
            day = int(day)*7
            dt = datetime.today() - timedelta(days=day)

        elif 'hours ago' in date_review or 'hour ago' in date_review:
            hours = date_review.replace('hours ago', '').replace('hour ago', '')
            dt = datetime.today() - timedelta(hours=int(hours))

        # format the date into 02-Jan-20
        new_datetime = dt.strftime("%d-%b-%y")
    else:
        new_datetime = date_review
        
    #spilt the sentence
    splitted_sentence= re.split(r'[.!]', current_review) 
    print(splitted_sentence)
    
    for j in range(len(splitted_sentence)):
        
        splitted_sentence[j].strip()
        if len(splitted_sentence[j]) > 0 :
            
            processed_review = splitted_sentence[j].strip()
            
            # Negation handling
            processed_review = processed_review.split()
            processed_review =[appos[w] if w in appos else w for w in processed_review]
            processed_review = " ".join(processed_review) 
            
            processed_review = processed_review.strip()
            # Remove all the special characters
            processed_review = re.sub(r'\W', ' ', processed_review)
            # Removing prefixed 'b'
            processed_review = re.sub(r'^b\s+', '', processed_review)
            # remove all single characters contains a white space character
            processed_review= re.sub(r'^[a-zA-Z]$', ' ', processed_review)
            # Substituting multiple spaces with single space
            processed_review = re.sub(r'\s+', ' ', processed_review, flags=re.I)
          
            
            processed = processed_review.strip()
            #print(processed)
            
            # final check if the processed_review is empty
            final = []
            if processed != "":
                tokenized = word_tokenize(processed)
                #removed additional letters eg "gooooooodddddd" to "good"
                for w in tokenized:
                    processed_2 = reduce_lengthening(w)
                    processed_2 = spell.correction(processed_2)
                    final.append(processed_2)
                final =' '.join(final)
                #print(final)
                # append into a dictionary
                platform_csv.append(platform_list[i])
                id_csv.append(row_id_list[i])
                brand_csv.append(brand_list[i])
                category_csv.append(category_list[i])
                product_name_csv.append(product_name_list[i])
                prices_csv.append(price_list[i])
                reviewer_csv.append(reviewer_list[i])
                review_csv.append(current_review)
                review_splitted_csv.append(final)
                product_variation_csv.append(product_variation_list[i])
                rating_csv.append(rating_list[i])
                date_review_csv.append(new_datetime)
                response_csv.append(response_list[i])


['delivery was quite fast, and item was on discount too so i couldn’t complain', ' so far liking this product', ' will continue to use it :)']
delivery was quite fast and item was on discount too so i couldn t complain
delivery was quite fast and item was on discount too so i could t complain
so far liking this product
so far liking this product
will continue to use it
will continue to use it
['received item sealed properly', '\r\r\r\nquick delivery too', '\r\r\r\nitem not tested yet as for gifting purpose', '']
received item sealed properly
received item sealed properly
quick delivery too
quick delivery too
item not tested yet as for gifting purpose
item not tested yet as for lifting purpose
['yet to try but delivery was prompt and product has long expiry', '']
yet to try but delivery was prompt and product has long expiry
yet to try but delivery was prompt and product has long expiry
['well wrapped,  in good condition,  super fast delivery,  first time buy,  yet to try,  thanks selle

['2nd round purchase after having quite good improvement from the first time usage of the essence', ' high quality product and fitted well to the asian facial skins', ' seller provided faster service and delivery', ' thanks seller and shopee', '']
2nd round purchase after having quite good improvement from the first time usage of the essence
and round purchase after having quite good improvement from the first time usage of the essence
high quality product and fitted well to the asian facial skins
high quality product and fitted well to the asian facial skins
seller provided faster service and delivery
seller provided faster service and delivery
thanks seller and shopee
thanks seller and hope
['#shopeehaul good product quality fast delivery excellent service by seller\r\r\r\nproducts were nicely packed and delivered', ' got a lot of free stuff too', ' thanks seller', ' ']
shopeehaul good product quality fast delivery excellent service by seller products were nicely packed and delivered

good authentic product came in nice packaging
good authentic product came in nice packaging
very nice
very nice

['quick and fast delivery', ' nice packed and sealed', ' can’t wait to try out the products', ' ']
quick and fast delivery
quick and fast delivery
nice packed and sealed
nice packed and sealed
can t wait to try out the products
can t wait to try out the products

['products received well and thanks for the 2 free lipsticks ']
products received well and thanks for the 2 free lipsticks
products received well and thanks for the 2 free lipsticks
['received in good condition', ' bought on sale, masks were free', ' tried on the mask and it felt hydrating', ' ']
received in good condition
received in good condition
bought on sale masks were free
bought on sale masks were free
tried on the mask and it felt hydrating
tried on the mask and it felt hydration

['received after 3days from order', ' hope the product doea what it claims']
received after 3days from order
received after days

good product quality fast delivery
['fast delivery, received with 2weeks', '', ' really worth the price, compare to shop is really much more cheaper and cone with the complexion', '', ' both manufacturers date are this year', '', ' thxqqq~~ ']
fast delivery received with 2weeks
fast delivery received with weeks
really worth the price compare to shop is really much more cheaper and cone with the complexion
really worth the price compare to shop is really much more cheaper and cone with the complexion
both manufacturers date are this year
both manufacturers date are this year
thxqqq
thxqq
['received the items in good condition\r\r\r\nexpiry in 2022 \r\r\r\nworth buying', '']
received the items in good condition expiry in 2022 worth buying
received the items in good condition expiry in 2022 worth buying
['fast delivery and item was well received second time purchase and it came with uv sunblock', ' love to see more promo good buy ']
fast delivery and item was well received second time pur

['received about 2-3 days after placing order, together with the sunscreen', ' used for a few days, pimples seems less angry now', ' hopefully will be able to live up to the good reviews i read online', ' ']
received about 2 3 days after placing order together with the sunscreen
received about 2 3 days after placing order together with the sunscreen
used for a few days pimples seems less angry now
used for a few days pimples seems less angry now
hopefully will be able to live up to the good reviews i read online
hopefully will be able to live up to the good reviews i read online

['fast delivery', ' is definitely cheaper than retails and come with free gift', ' can’t commend much of the effect as only 2nd day i used, hope the product works well', '']
fast delivery
fast delivery
is definitely cheaper than retails and come with free gift
is definitely cheaper than retails and come with free gift
can t commend much of the effect as only 2nd day i used hope the product works well
can t com

...

tried it on a wedding dinner on my second day and surprisingly it does look a bit cakey but not too much and acceptable
tried it on a wedding dinner on my second day and surprisingly it does look a bit cakey but not too much and acceptable
what i really like about this powder is it really does stay matte through out the day and my face do not look anymore shining
what i really like about this powder is it really does stay matte through out the day and my face do not look anymore shining
['this powder does what it says, it’s long lasting throughout the day and stays matt on my face', ' i also liked that it’s light and not obviously thick and uncomfortable on the face', ' the only thing i wish was inproved would maybe be the sponge, it didn’t feel like it was good and was a little difficult to use cos of how large and wide it was', '']
this powder does what it says it s long lasting throughout the day and stays matt on my face
this powder does what it says it s long lasting throughout th

highly recommended product
highly recommended product
affordable too so no harm trying
affordable too so no harm trying
['i am surprised by how light weighted this powder is', ' it does not create those "white sesame" appearance on my pores', ' i used to touch up my t-zone few times a day but with this powder, i only need to touch up once', ' although it is matte, it does not dried up my face and caused it to flake', ' i highly recommend to those who frequently need to touch up', '']
i am surprised by how light weighted this powder is
i am surprised by how light weighted this powder is
it does not create those white sesame appearance on my pores
it does not create those white sesame appearance on my pores
i used to touch up my t zone few times a day but with this powder i only need to touch up once
i used to touch up my t zone few times a day but with this powder i only need to touch up once
although it is matte it does not dried up my face and caused it to flake
although it is matte i

['the matte powder is very compact and easy to keep in your bag', ' it gives a matte finish and sits nicely on prime face', ' the colour selections is limited but you can still make it work with your foundation or concealer', '']
the matte powder is very compact and easy to keep in your bag
the matte powder is very compact and easy to keep in your bag
it gives a matte finish and sits nicely on prime face
it gives a matte finish and sits nicely on prime face
the colour selections is limited but you can still make it work with your foundation or concealer
the colour selections is limited but you can still make it work with your foundation or concealer
["l’oreal paris infallible pro-matte powder blends well with my skin and doesn't feel cakey at all", ' i have used it alone as a foundation and it felt great', ' i do not need to touch up throughout the day', ' will definitely recommend this product to my friends because it is so much better than most of the other brands of matte powder in 

the powder is fine and soft to the skin
i was using true match in the past and still prefer that range
i was using true match in the past and still prefer that range
['i chose no', '10 and the colour is suitable for my skin', " i'm usually a n21 for bb cushion", ' \r\r\n\r\r\npowder is airy, smooth and matte', " it's also very natural and provide some coverage", ' can be use by itself on good skin days', ' \r\r\n\r\r\ndesign is very compact and lightweight', ' comes with mirror', ' \r\r\n\r\r\nthe only downside for me is the puff', ' other than being able to apply the powder as per normal and it is soft, the design of the puff looks like those once off disposable ones', '  thus it makes me doubt the quality and durability', " i feel it didn't matched the brand image and hope it can be improved", ' \r\r\n\r\r\ni see myself buying and will recommend this powder to friends and family', '']
i chose no
i chose no
10 and the colour is suitable for my skin
10 and the colour is suitable for my

['surely a matt foundation for a drug store brand', ' will repurchase again']
surely a matt foundation for a drug store brand
surely a matt foundation for a drug store brand
will repurchase again
will repurchase again
['it leaves your skin feeling smooth and dry throughout the day', ' no need for touch up at all', '']
it leaves your skin feeling smooth and dry throughout the day
it leaves your skin feeling smooth and dry throughout the day
no need for touch up at all
no need for touch up at all
['its gentle on skin and i had a good experience using it', '']
its gentle on skin and i had a good experience using it
its gentle on skin and i had a good experience using it
['it instantly makes skin look matte without looking cakey or too powdery', '']
it instantly makes skin look matte without looking cakey or too powdery
it instantly makes skin look matte without looking cakey or too powdery
['no need to touch up after few hour no more oily skin it suit my skin']
no need to touch up after f

In [ ]:
#Store into a new data frame
splitted_data = {'ID':id_csv,'Platform':platform_csv ,'Brand':brand_csv, 'Category': category_csv, 'Product Name ': product_name_csv, 'Price':prices_csv ,'Reviewer':reviewer_csv,'Review':review_csv, 'Review_splitted':review_splitted_csv, 'Product Purchase':product_variation_csv,'Ratings':rating_csv,'Date Of Review':date_review_csv,'Response': response_csv }
splitted_df = pd.DataFrame.from_dict(splitted_data)
splitted_df['Date Of Review'] = pd.to_datetime(splitted_df['Date Of Review'])
splitted_df.head()
splitted_df.info()

#splitted_df.to_csv('Lazada_process_v1.csv')

### Create the Training Data Set

In [ ]:
# spacy for lemmatization
import spacy

def lemmatization(texts):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = sp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc ])
    return texts_out

In [ ]:
from nltk.corpus import stopwords
from nltk.corpus import PlaintextCorpusReader


review_cleaned=[]
stop_list = stopwords.words('english')


stemmer = PorterStemmer()
sp = spacy.load('en', disable=['parser', 'ner'])

splitted_df= splitted_df[splitted_df.Review_splitted != 'no comments review is an image']
tokenized_review = splitted_df["Review_splitted"]


for sentence in tokenized_review:
    tokenized_review = nltk.word_tokenize(sentence)
    review_stopremoved = [w for w in tokenized_review if w not in stop_list]
#     review_translate = [gs.translate(w,'en') for w in review_stopremoved if re.findall(r'[\u4e00-\u9fff]+', w)]
#     print(review_translate)
    review_cleaned.append(review_stopremoved)

# Lemmatized all the words
data_lemmatized = lemmatization(review_cleaned)
#print(data_lemmatized)


splitted_df['tokenized_review'] = data_lemmatized    
splitted_df.head()

In [ ]:
## translator
import goslate

text = "价格合理"

gs = goslate.Goslate()
translatedText = gs.translate(text,'en')
print(translatedText)

### Import the Positive , Negative , Increment , Decrement and Inverse Words


In [ ]:
import nltk 

pos_lexicon = 'dict/positive-words.txt'
neg_lexicon = 'dict/negative-words.txt'
inc_words = 'dict/increment-words.txt'
dec_words = 'dict/decrement-words.txt'
inv_words = 'dict/inverse-words.txt'


# Read the positive sentiment lexicon.
pos_dict = {}
f = open(pos_lexicon, 'r', encoding = "ISO-8859-1")
for line in f:
    line = line.strip()
    pos_dict[line] = 1
f.close()

# Read the negative sentiment lexicon.
neg_dict = {}
f = open(neg_lexicon, 'r', encoding = "ISO-8859-1")
for line in f:
    line = line.strip()
    neg_dict[line] = 1
f.close()


# Read the increment words.
inc_dict = {}
f = open(inc_words, 'r', encoding = "ISO-8859-1")
for line in f:
    line = line.strip()
    inc_dict[line] = 1
f.close()

# Read the decrement words.
dec_dict = {}
f = open(dec_words, 'r', encoding = "ISO-8859-1")
for line in f:
    line = line.strip()
    dec_dict[line] = 1
f.close()

# Read the inverse words.
inv_dict = {}
f = open(inv_words, 'r', encoding = "ISO-8859-1")
for line in f:
    line = line.strip()
    inv_dict[line] = 1
f.close()


### Lexicon-based Sentiment Polarity Classification

In [ ]:
predicted_labels = []
score_label=[]

for sent in data_lemmatized:
    total_score = 0
   # print(sent)
    for w in sent:
        score = 0
        if w in pos_dict:
            score = 1
        elif w in neg_dict:
            score = -1
        #check previous word
        position = sent.index(w)
        previous_word = sent[position-1]
        if previous_word is not None:
            if previous_word in inc_dict : 
                score *= 2.0
            elif previous_word in dec_dict:
                score /=2
            elif previous_word in inv_dict:
                score *= -1  
        total_score+=score
    
    if total_score >0:
        predicted_labels.append('1')
        
    elif total_score <0:
        predicted_labels.append('-1')
    else:
        predicted_labels.append('0')
        
    score_label.append(int(total_score))
    #print(total_score)

splitted_df["Polarity"] = predicted_labels
splitted_df["Score"] = score_label
splitted_df.to_csv('Lazada_sentiment_1.csv')


### Combine Polarity for Splitted Review

In [ ]:
from statistics import mode

#find the mode of each review
result = splitted_df.groupby('ID')['Polarity'].agg(lambda x:x.value_counts().index[0])
#print(result)

# insert a overall polarity col, with the mode value
splitted_df['Overall Polarity'] = [result[id] for id in splitted_df['ID']]